## Imports

In [ ]:
import cv2
from matplotlib import pyplot as plt
from lane_detection import Line
from sklearn.cluster import DBSCAN
import lane_detection
import lane_following
import numpy as np

## Helper Functions

In [ ]:
def show(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

## Read Image

In [ ]:
# frame = cv2.imread("frames/frame1320.jpg")
frame = cv2.imread("frames/frame660.jpg")
show(frame)

## Slice Image

In [ ]:
# sliced_frame = frame[ int(frame.shape[0] / 2) : frame.shape[0]]
sliced_frame = frame[int(frame.shape[0] / 2) : frame.shape[0]]
show(sliced_frame)

## Convert to Grayscale

In [ ]:
gray = cv2.cvtColor(sliced_frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale
show(gray)


## Blurring

In [ ]:
blur = cv2.GaussianBlur(gray, (19, 19), 0)
show(blur)

## Black and White Conversion

In [ ]:
_, bw_image = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY)
show(bw_image)

## Erosion

In [ ]:
eroded = cv2.erode(bw_image, (9, 9), iterations = 10)
show(eroded)

## Edge Detection

In [ ]:
edges = cv2.Canny(
    eroded, 20, 100, apertureSize=3
)  # detect edges
show(edges)


## Hough Lines to get Lines
Edge detection just highlights the edges, HoughLinesP gives us the points describing important edges

In [ ]:
lines = cv2.HoughLinesP(
    edges,
    rho=1,
    theta=np.pi / 180,
    threshold=100,
    minLineLength=100,
    maxLineGap=20,
)  # detect lines

lines = [Line(line[0][0], line[0][1], line[0][2], line[0][3]) for line in lines]
# lines = lane_detection.merge_colinear_lines(lines)
print(f"{len(lines) = }")
drawn = lane_detection.draw_lines(sliced_frame, lines)

show(drawn)


## Group Lines by Slope
Using DBSCAN, group the lines

In [ ]:
from lane_detection import draw_lines
slopes = [line.slope for line in lines]
slopes = np.array(slopes).reshape(-1, 1)  # convert slopes to a 2d array

slope_tolerance = 0.1
dbscan = DBSCAN(eps=slope_tolerance, min_samples=2)
labels = dbscan.fit_predict(slopes)  # labels is a list of clusters, basically

def group(labels: list[int], data: list[Any]) -> dict[int, Any]:
    grouped_data = {}
    for index, element in enumerate(data):
        label = labels[index]
        if label not in grouped_data:
            grouped_data[label] = []
        grouped_lines[label].append(element)
# Group lines based on the cluster labels
grouped_lines = {}
for idx, line in enumerate(lines):
    label = labels[idx]
    if label not in grouped_lines:
        grouped_lines[label] = []
    grouped_lines[label].append(line)

for group in grouped_lines:
    show(draw_lines(sliced_frame, grouped_lines[group], random=True))
        

## Group and Combine Lines by X-Intercept
Now we have a dictionary pairing all of the lines with similar slopes together. However, this doesn't work great for all cases. Sometimes you have lines with similar slopes, but that are not actually colinear; that is, they are parallel but not intersecting. 
Because of such a case, we must first determine whether the lines share similar x-intercepts before we can merge them.

In [ ]:
merged_lines = []

for label, lines in grouped_lines.items():
    print(label, lines)

In [ ]:
import math
def detect_lanes(lines: list[Line], height: int = 1080, width: int = 1920, center_lane_tol = 0.5, parallel_tol = 0.5, x_intercept_tol = 250) -> list[tuple[Line, Line]]:
    center = width / 2
    lanes = []
    lines.sort(key=lambda x: x.x_intercept)

    for i in range(len(lines[:-1])):
        line1 = lines[i]
        for j in range(i+1, len(lines[:-1])):
            line2 = lines[j]
            if line1.is_paired() or line2.is_paired():
                print("lines are paired")
                break

            if math.isclose(line1.slope, -1 * line2.slope, rel_tol=center_lane_tol):
                # lines are a lane near the center
                line1.paired = True
                line2.paired = True
                lanes.append((line1, line2))
                break # line 1 has a match with line 2, so pick a new line 1

            elif math.isclose(line1.slope, line2.slope, rel_tol=parallel_tol):
                # slopes are close to parallel
                if math.isclose(
                    line1.x_intercept, line2.x_intercept, rel_tol=x_intercept_tol
                ):
                    # x-intercepts are close
                    if ((line1.x_intercept > center) and (line2.x_intercept > center)) or (
                        (line1.x_intercept < center) and (line2.x_intercept < center)
                    ):
                        # the lines are probably a pair
                        line1.paired = True
                        line2.paired = True
                        lanes.append((line1, line2))
                        break # found a pair, so start a new
    return lanes

In [ ]:
lanes = detect_lanes(lines, sliced_frame.shape[0], sliced_frame.shape[1])
laned_img = lane_detection.draw_lanes(sliced_frame, lanes)
plt.imshow(cv2.cvtColor(laned_img, cv2.COLOR_BGR2RGB))

(center_slope, center_intercept) = lane_following.get_lane_center(lanes)

direction = lane_following.recommend_direction(center_intercept, center_slope, sliced_frame.shape[1])

print(f"The AUV should go to the {direction}")

In [ ]:

# cap = cv2.VideoCapture('AUV_Vid.mkv')

# count = 0 # the number of frames since the last
# frequency = 60 # the number of frames to skip

# while cap.isOpened():
#     ret, frame = cap.read()
#     if count % frequency == 0:
#         cv2.imwrite(f"./frames/frame{count}.jpg", frame)
#         # plt.imshow(cv2.cvtColor(render_frame(frame), cv2.COLOR_BGR2RGB))
#         # plt.show()
        
#     count += 1